In [10]:
#导入库以及读入数据集
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

raw=pd.read_csv('exp3.csv')
raw.head()

,index,queueId,mapId,seasonId,team1_win,team1_firstBlood,team1_firstTower,team1_firstInhibitor,team1_firstBaron,team1_firstDragon,...,player9_goldEarned,player9_role,player9_lane,player10_championId,player10_kills,player10_deaths,player10_assists,player10_goldEarned,player10_role,player10_lane
0,0,420,11,13,0,True,False,False,False,False,...,6967,DUO_SUPPORT,NONE,157,3,0,8,8168,DUO,NONE
1,1,420,11,13,1,False,True,True,False,True,...,8133,SOLO,MIDDLE,555,3,7,4,6943,DUO_SUPPORT,BOTTOM
2,2,420,11,13,0,False,True,False,False,True,...,11868,DUO_CARRY,BOTTOM,122,2,2,1,9685,SOLO,TOP
3,3,420,11,13,0,True,False,False,False,True,...,6703,DUO_SUPPORT,BOTTOM,23,6,2,10,13109,SOLO,TOP
4,4,420,11,13,1,True,True,True,False,False,...,6823,DUO,MIDDLE,25,5,7,8,6809,DUO_SUPPORT,BOTTOM


In [11]:
#分别计算队伍经济差、K、D、A的差以及KDA差（KDA=[K+A]/D)
def team_gold():
    raw1=df.loc[:,['player1_goldEarned','player2_goldEarned','player3_goldEarned','player4_goldEarned','player5_goldEarned']]
    raw2=df.loc[:,['player6_goldEarned','player7_goldEarned','player8_goldEarned','player9_goldEarned','player10_goldEarned']]
    win=df.loc[:,'team1_win']
    raw1['gold1'] = raw1.apply(lambda x: x.sum(), axis=1)
    raw2['gold2'] = raw2.apply(lambda x: x.sum(), axis=1)
    gold=pd.concat([raw1['gold1'],raw2['gold2'],win],axis=1)
    gold['diff'] = gold['gold1']-gold['gold2']
    return gold
    
def team_KDA(df):
    raw1=df.iloc[:,12:43]
    raw2=df.iloc[:,44:78]
    win=df.loc[:,'team1_win']
    raw1['t1_kills']=raw1['player1_kills']+raw1['player2_kills']+raw1['player3_kills']+raw1['player4_kills']+raw1['player5_kills']
    raw1['t1_deaths']=raw1['player1_deaths']+raw1['player2_deaths']+raw1['player3_deaths']+raw1['player4_deaths']+raw1['player5_deaths']
    raw1['t1_assists']=raw1['player1_assists']+raw1['player2_assists']+raw1['player3_assists']+raw1['player4_assists']+raw1['player5_assists']
    raw2['t2_kills']=raw2['player6_kills']+raw2['player7_kills']+raw2['player8_kills']+raw2['player9_kills']+raw2['player10_kills']
    raw2['t2_deaths']=raw2['player6_deaths']+raw2['player7_deaths']+raw2['player8_deaths']+raw2['player9_deaths']+raw2['player10_deaths']
    raw2['t2_assists']=raw2['player6_assists']+raw2['player7_assists']+raw2['player8_assists']+raw2['player9_assists']+raw2['player10_assists']
    
    KDA=pd.concat([raw1[['t1_kills','t1_deaths','t1_assists']],raw2[['t2_kills','t2_deaths','t2_assists']],win],axis=1)
    KDA['t1win']=df['team1_win']
    KDA['t1_KDA']=(KDA['t1_kills']+KDA['t1_assists'])/KDA['t1_deaths']
    KDA['t2_KDA']=(KDA['t2_kills']+KDA['t2_assists'])/KDA['t2_deaths']
    KDA['KDAdiff']=KDA['t1_KDA']-KDA['t2_KDA']
    #KDA['killdiff']=KDA['t1_kills']-KDA['t2_kills']
    #KDA['deathdiff']=KDA['t1_deaths']-KDA['t2_deaths']
    #KDA['assistdiff']=KDA['t1_assists']-KDA['t2_assists']
    return KDA[['KDAdiff']]

In [12]:
#预处理并提取特征（如一血、一龙、经济差等）
def geten(df):
    en=pd.DataFrame()
    en=df.iloc[:,4:11].astype('int')
    gold=team_gold()
    KDA=team_KDA(df)
    gold=(gold-gold.min())/(gold.max()-gold.min())
    KDA=(KDA-KDA.min())/(KDA.max()-KDA.min())
    en['golddiff']=gold['diff']
    en=pd.concat([en,KDA],axis=1)
    en=en.dropna(axis=0,how='any')
    
    en.loc[en.team1_win == 0,'team1_win']=-1 #把失败的对局改为-1，以方便确定投票结果
    
    en=en.reset_index(drop=True)
    return en   

In [13]:
#KNN算法实现：计算特征距离，选取k个邻居并投票
def KNN(train,test,k):
    KNNs=time.time()
    correct=0
    num=len(test)
    test['pre']=0
    for j in tqdm(range(num)):
        train['distance']=0
        for i in range(len(par)):
            train['distance']=train['distance']+(train.iloc[:,i+1]-test.iloc[j,i+1])**2*par[i]
        
        distance=train['distance']**0.5
        distance=distance.sort_values(ascending=True)
        distance=distance.iloc[0:k]           #截断前k近的邻居
        
        sum=0                                 #sum进行计票
        judger=train.loc[distance.index]
        sum=judger['team1_win'].sum()
        if sum > 0:
            test.loc[j,'pre']=1
        else:
            test.loc[j,'pre']=-1
    
    for k in range(num):
        if test.loc[k,'team1_win'] == test.loc[k,'pre']:
            correct=correct+1
            
    rate=correct/num
    KNNe=time.time()
    print('预测准确率是%.6f KNN用时：'%rate,KNNe-KNNs)
    return rate

In [14]:
pd.options.mode.use_inf_as_na = True   #处理异常值
#参数表
par=[1,1,4,1,2,2,480,120]
k=30
df=raw.copy()

In [15]:
test=geten(df.loc[0:16000])
#train=geten(pd.concat([df.loc[32001:],df.loc[0:16000]],axis=0))
train=geten(df.loc[16001:])
KNN(train,test,k)

100%|████████████████████████████████████████████████████████████████████████████| 15839/15839 [04:16<00:00, 61.68it/s]


预测准确率是0.981312 KNN用时： 257.0684072971344


0.9813119515120904

In [16]:
test=geten(df.loc[16001:32000])
train=geten(pd.concat([df.loc[32001:],df.loc[0:16000]],axis=0))
KNN(train,test,k)

100%|████████████████████████████████████████████████████████████████████████████| 15841/15841 [04:22<00:00, 60.27it/s]


预测准确率是0.978852 KNN用时： 263.06513476371765


0.9788523451802286

In [17]:
test=geten(df.loc[32001:48000])
train=geten(pd.concat([df.loc[48001:],df.loc[0:32000]],axis=0))
KNN(train,test,k)

100%|████████████████████████████████████████████████████████████████████████████| 15835/15835 [04:11<00:00, 62.87it/s]


预测准确率是0.980044 KNN用时： 252.14181351661682


0.980044205873066

In [18]:
test=geten(df.loc[48001:64000])
train=geten(pd.concat([df.loc[64001:],df.loc[0:48000]],axis=0))
KNN(train,test,k)

100%|████████████████████████████████████████████████████████████████████████████| 15848/15848 [04:14<00:00, 62.20it/s]


预测准确率是0.980313 KNN用时： 255.012300491333


0.9803129732458354

In [19]:
test=geten(df.loc[64001:])
train=geten(df.loc[0:64000])
KNN(train,test,k)

100%|████████████████████████████████████████████████████████████████████████████| 15824/15824 [04:12<00:00, 62.57it/s]


预测准确率是0.979778 KNN用时： 253.15547800064087


0.9797775530839231

In [ ]:
The average is 0.98001139919079806